## Example geochemical calculations using data logger data 

In [ ]:
#Check whether we are running on Colab or locally.
try:
    import google.colab
    IN_COLAB = True
    base_path = 'https://raw.githubusercontent.com/CovingtonResearchGroup/olm-examples/dataloggers/'
except:
    IN_COLAB = False
    base_path = './'
print('Base working path for data files is',base_path)

In [ ]:
#If olm isn't already installed (or if you're running in Colab), then run this cell of code.
!pip install olm-karst

In [ ]:
#We will run in pylab mode, to import plotting functions.
%pylab inline

In [ ]:
from olm.loggers.CampbellToolkit import read_dat
#Read in water quality data
Langle_water = read_dat(base_path + 'data/CR800_Langle_Water.dat')
Langle_water.head()

In [ ]:
#Read in dissolved CO2 data
Langle_CO2 = read_dat(base_path + 'data/CR800_Langle_CarbonDioxide.dat')
Langle_CO2.head()

#### Join and resample dataframes

In [ ]:
from olm.loggers.loggerScripts import joinAndResampleLoggers
Langle = joinAndResampleLoggers([Langle_water, Langle_CO2], '1H', suffixes=['water','CO2'])
Langle.head()

#### Converting raw conductivity to specific conductance at 25 C

In [ ]:
from olm.general import condTo25

#Filter out bad values during download
Langle['2013-05-25 18:00':'2013-05-25 19:00'] = np.nan
Langle = Langle.dropna()
#Convert raw conductivity muS/cm
Cond_raw = 1000* Langle.Ct_Avg_water 
T_C = Langle.Temp_C_c_Avg_water

#Calculate temperature corrected SpC
SpC = condTo25(Cond_raw,T_C)
Cond_raw.plot()
SpC.plot()
ylabel(r'Conductivity ($\mu S/cm$)')
legend(['Raw cond', 'SpC at 25 C']);

#### Estimating Ca hardness from SpC

In [ ]:
from olm.general import CaFromCond
Ca = CaFromCond(SpC, T_C=T_C)
#Note this function uses a global relation between hardness and SpC. A local relation
#for the site will be more accurate (though still an estimate).
Ca.plot()
ylabel('Estimated Ca (mg/L)')
#Add Ca column into Langle DataFrame


#### Creating a Series of solution objects

In [ ]:
from olm.loggers.loggerScripts import joinAndResampleLoggers
from olm.calcite import solutionFromCaPCO2Relaxed
from olm.general import mgL_to_molL

#Convert raw voltage value to pCO2 in atm
volts_to_ppm = 10
ppm_to_atm = 1e-6
CO2 = volts_to_ppm*ppm_to_atm*Langle.CO2_Avg_CO2
#Convert Ca to mol/L
Ca_mol_L = mgL_to_molL(Ca, 'Ca')
Langle_sols = solutionFromCaPCO2(Ca_mol_L, CO2, T_C=T_C)


In [ ]:
#Now we have a Pandas Series of solution objects. We can use this to do lots of cool things.
Langle_sols.head()

#### Calculate equilibrium Ca concentration w.r.t. Calcite for entire solution series

In [ ]:
from olm.calcite import concCaEqFromSolution
CaEq = concCaEqFromSolution(Langle_sols)
CaEq.plot()
ylabel('Equilibrium Ca (mol/L)')

#### Calculate saturation ratio ($\rm{[Ca]/[Ca_{eq}]}$)

In [ ]:
Sat_ratio = Ca_mol_L/CaEq
Sat_ratio.plot()
axhline(y=1, linestyle='--',color='k')
ax = gca()
text(0.75,0.85, 'Saturation line', transform=ax.transAxes)
ylabel(r'$\rm{[Ca]/[Ca_{eq}]}$');

#### Estimate calcite dissolution rates

In [ ]:
from olm.calcite import palmerFromSolution
R = palmerFromSolution(Langle_sols, rho=2.6, impure=True)
R.plot()
ylabel('Dissolution rate (mm/yr)');